In [1]:
import os
import datetime
from fredapi import Fred
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd

load_dotenv(".env")
FRED_API_KEY = os.getenv("API_KEY")
fred = Fred(api_key=FRED_API_KEY)
client = MongoClient('mongodb://127.0.0.1:27017/')
db = client['chartData']

In [12]:
# Define your series dictionary, start and end dates as before
series_dict = {
    'bonds2tr': ('DGS2', 'M'),
    'gdpworld': ('NYGDPMKTPCDWLD', 'A'),
    'gdp': ('GDP', 'Q'),
    'recession': ('JHDUSRGDPBR', 'Q'),
    'bond10tr': ('DGS10', 'M'),
    'ppi': ('PPIACO', 'M'),
    'cpi': ('CPIAUCSL', 'M'),
    'unrate': ('UNRATE', 'M'),
    'debt': ('GFDEBTN', 'Q'),
    'fedrate': ('FEDFUNDS', 'M'),
    'm0': ('BOGMBASE', 'M'),
    'm3': ('MABMM301USM189S', 'M'),
    'cbasstogdp': ('DDDI06USA156NWDB', 'A'),
    'resins': ('TOTRESNS', 'M'),
    'oil': ('WTISPLC', 'M'),
    'indpro': ('INDPRO', 'M'),
    'houses': ('MSPUS', 'Q'),
    'cp': ('CP', 'Q')
}

# Define dates
start_date = datetime.datetime(1970, 1, 1)
end_date = datetime.datetime(2022, 12, 31)

# Initialize DataFrames
dfs = {'M': pd.DataFrame(), 'Q': pd.DataFrame(), 'A': pd.DataFrame()}

# Function to fetch and merge series data
def fetch_and_merge(df, var_name, series_id, start, end):
    series_data = fred.get_series(series_id)
    series_data = series_data.loc[start_date:end_date]
    series_df = series_data.to_frame(name=var_name)
    return df.join(series_df, how='inner') if not df.empty else series_df


# Loop through each series
for var_name, (series_id, freq) in series_dict.items():
    dfs[freq] = fetch_and_merge(dfs[freq], var_name, series_id, start_date, end_date)

In [14]:
dfs['Q']

,gdp,recession,debt,houses,cp
1970-01-01,1051.200,1.0,372007.0,23900.0,55.323
1970-04-01,1067.375,1.0,370094.0,24400.0,55.194
1970-07-01,1086.059,1.0,378678.0,23000.0,56.132
1970-10-01,1088.608,1.0,389158.0,22600.0,53.350
1971-01-01,1135.156,0.0,391668.0,24300.0,60.152
...,...,...,...,...,...
2021-10-01,24654.603,0.0,29617215.0,423600.0,2847.819
2022-01-01,25029.116,0.0,30400960.0,433100.0,2909.892
2022-04-01,25544.273,0.0,30568582.0,449300.0,3148.734
2022-07-01,25994.639,0.0,30928912.0,468000.0,3013.486
